## Setting Up Azure OpenAI Client

1. **AzureOpenAI Client**:
   - The `AzureOpenAI` client is initialized using the Azure OpenAI API key, version, and endpoint.
   - This client allows us to interact with GPT-4 hosted on Azure for various tasks such as query rewriting, generating step-back queries, and query decomposition.
2. **Prompts from LangChain**:
   - `PromptTemplate` is used to structure the prompt format when communicating with the Azure LLM.


## Setting Up Azure OpenAI Client

1. **AzureOpenAI Client**:
   - The `AzureOpenAI` client is initialized using the Azure OpenAI API key, version, and endpoint.
   - This client allows us to interact with GPT-4 hosted on Azure for various tasks such as query rewriting, generating step-back queries, and query decomposition.
2. **Prompts from LangChain**:
   - `PromptTemplate` is used to structure the prompt format when communicating with the Azure LLM.


In [1]:
# Cell 1: Import libraries and set up Azure OpenAI client

from dotenv import load_dotenv
import os
from openai import AzureOpenAI
from langchain.prompts import PromptTemplate



In [2]:
load_dotenv('variables.env')
# Azure OpenAI configuration
azure_openai_api_key = os.getenv('AZURE_OPENAI_API_KEY')
azure_openai_api_version = os.getenv('AZURE_OPENAI_API_VERSION')
azure_endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')

print(azure_openai_api_key)
print(azure_openai_api_version)
print(azure_endpoint)

4d69d52da8164fc8939c74eae4d66ef9
2024-02-15-preview
https://home-openai-01.openai.azure.com/


In [3]:
# Set up Azure OpenAI client
azure_client = AzureOpenAI(
    api_key=azure_openai_api_key,
    api_version=azure_openai_api_version,
    azure_endpoint=azure_endpoint
)

## Query Rewriting Function

1. **LLM Call**:
   - The `azure_llm_call()` function sends a prompt to GPT-4 using Azure OpenAI to generate a response.
   - The `temperature=0` ensures deterministic output by limiting randomness in the response.

2. **Query Rewriting Prompt**:
   - The `query_rewrite_template` defines the structure of the prompt that will be used to ask GPT-4 to rewrite a query.
   - The rewritten query should be more specific to improve retrieval quality in a RAG system.


In [4]:
# Initialize the LLM for query rewriting
def azure_llm_call(prompt, model="gpt-4o"):
    response = azure_client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )
    return response.choices[0].message.content

# Create a prompt template for query rewriting
query_rewrite_template = """You are an AI assistant tasked with reformulating user queries to improve retrieval in a RAG system. 
Given the original query, rewrite it to be more specific, detailed, and likely to retrieve relevant information.

Original query: {original_query}

Rewritten query:"""

# Define the prompt template for query rewriting
query_rewrite_prompt = PromptTemplate(
    input_variables=["original_query"],
    template=query_rewrite_template
)

# Define the function to rewrite the query
def rewrite_query(original_query):
    prompt = query_rewrite_prompt.format(original_query=original_query)
    return azure_llm_call(prompt)


## Demonstrating Query Rewriting

1. **Original Query**:
   - A sample query is provided, asking about the impacts of climate change.

2. **Rewriting Process**:
   - The `rewrite_query()` function is applied to the original query, making it more specific.
   - The results (original and rewritten queries) are printed.


In [5]:
# Example query over the "Understanding Climate Change" dataset
original_query = "What are the impacts of climate change on the environment?"

# Apply the query rewriting function to improve the query
rewritten_query = rewrite_query(original_query)

# Print the original and rewritten queries
print("Original query:", original_query)
print("\nRewritten query:", rewritten_query)


Original query: What are the impacts of climate change on the environment?

Rewritten query: How does climate change affect various aspects of the environment, such as biodiversity, sea levels, weather patterns, and ecosystems?


## Step-back Prompting Function

1. **Step-back Query**:
   - A step-back query broadens the original question to retrieve more general context or background information, which can be useful in providing a comprehensive answer.


In [6]:
# Create a prompt template for step-back prompting
step_back_template = """You are an AI assistant tasked with generating broader, more general queries to improve context retrieval in a RAG system.
Given the original query, generate a step-back query that is more general and can help retrieve relevant background information.

Original query: {original_query}

Step-back query:"""

# Define the prompt template for step-back prompting
step_back_prompt = PromptTemplate(
    input_variables=["original_query"],
    template=step_back_template
)

# Define the function to generate a step-back query
def generate_step_back_query(original_query):
    prompt = step_back_prompt.format(original_query=original_query)
    return azure_llm_call(prompt)


## Demonstrating Step-back Prompting

1. **Step-back Query**:
   - The original query is processed into a broader, more general query to provide a wider context.
   - The function `generate_step_back_query()` is applied, and both the original and the step-back query are printed.


In [7]:
# Apply the step-back prompting function to generate a broader query
step_back_query = generate_step_back_query(original_query)

# Print the original and step-back queries
print("Original query:", original_query)
print("\nStep-back query:", step_back_query)


Original query: What are the impacts of climate change on the environment?

Step-back query: What are the general effects of climate change?


## Sub-query Decomposition Function

1. **Sub-query Decomposition**:
   - This function breaks down a complex query into smaller sub-queries, which are easier to handle.
   - For example, a question about the effects of climate change could be broken down into several sub-questions about different aspects, such as biodiversity, agriculture, etc.


In [8]:
# Create a prompt template for sub-query decomposition
subquery_decomposition_template = """You are an AI assistant tasked with breaking down complex queries into simpler sub-queries for a RAG system.
Given the original query, decompose it into 2-4 simpler sub-queries that, when answered together, would provide a comprehensive response to the original query.

Original query: {original_query}

example: What are the impacts of climate change on the environment?

Sub-queries:
1. What are the impacts of climate change on biodiversity?
2. How does climate change affect the oceans?
3. What are the effects of climate change on agriculture?
4. What are the impacts of climate change on human health?"""

# Define the prompt template for sub-query decomposition
subquery_decomposition_prompt = PromptTemplate(
    input_variables=["original_query"],
    template=subquery_decomposition_template
)

# Define the function to decompose a query into sub-queries
def decompose_query(original_query: str):
    prompt = subquery_decomposition_prompt.format(original_query=original_query)
    response = azure_llm_call(prompt)
    sub_queries = [q.strip() for q in response.split('\n') if q.strip() and not q.strip().startswith('Sub-queries:')]
    return sub_queries


## Demonstrating Sub-query Decomposition

1. **Sub-query Generation**:
   - The original query is broken down into several sub-queries using `decompose_query()`.
   - These sub-queries are printed and can be used in the retrieval process to improve the response to the original question.


In [9]:
# Decompose the original query into simpler sub-queries
sub_queries = decompose_query(original_query)

# Print the resulting sub-queries
print("\nSub-queries:")
for i, sub_query in enumerate(sub_queries, 1):
    print(f"{i}. {sub_query}")



Sub-queries:
1. Original query: What are the impacts of climate change on the environment?
2. 1. How does climate change affect biodiversity and ecosystems?
3. 2. What are the impacts of climate change on oceanic conditions and marine life?
4. 3. How does climate change influence weather patterns and extreme weather events?
5. 4. What are the effects of climate change on land use and agriculture?
